In [13]:
import os
# ローカル環境の設定
os.environ['AWS_GLUE_CATALOG_FAKE_REGION'] = 'us-east-1'
os.environ['DISABLE_AWS_GLUE'] = 'true'
os.environ['DISABLE_SSL'] = 'true'

# 環境のセットアップ
from pyspark import SparkContext
from pyspark.sql import SparkSession
from awsglue.context import GlueContext
from pyspark.sql.functions import col, upper, concat, lit

print("=== 1. Spark環境のセットアップ ===")
# 既存のSparkContextを使用または新規作成
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = SparkSession.builder \
    .appName("DataExploration") \
    .enableHiveSupport() \
    .getOrCreate()
print("Spark環境の準備完了")

print("\n=== 2. CSVファイル直接読み込み ===")

# CSVファイルを直接読み込む
csv_path = "/home/glue_user/workspace/data/warehouse/test_table/test_table.csv"
print(f"読み込みパス: {csv_path}")
df = spark.read.option("header", "true").option("inferSchema", "true").csv(csv_path)
print("CSVファイルの読み込み完了")

print("\n=== 3. データの確認 ===")
print("読み込んだデータ:")
df.show()
df.printSchema()

print("\n=== 4. データ変換例 ===")
# データ変換の例 - 文字列操作
print("文字列操作による変換:")
transformed_df = df.withColumn("full_info", 
                              concat(col("name"), lit(" ("), col("department"), lit(")")))
transformed_df.show()

# 部門コードを追加
print("部門コードを追加:")
dept_code_df = df.withColumn("department_code", col("department").substr(0, 1))
dept_code_df.show()

print("\n=== 5. データの集計 ===")
from pyspark.sql.functions import count, avg

# 部署ごとの集計
agg_df = df.groupBy("department").agg(
    count("*").alias("count"), 
    avg("age").alias("avg_age")
)
print("部署ごとの集計:")
agg_df.show()

print("\n=== 6. テスト用データの結合 ===")
# テスト用のデータフレームを作成
dept_data = [("開発部", "東京"), ("営業部", "大阪"), ("総務部", "名古屋"), ("人事部", "東京")]
dept_df = spark.createDataFrame(dept_data, ["department", "location"])

# 結合
joined_df = df.join(dept_df, on="department")
print("結合結果:")
joined_df.show()

print("\n=== 7. 結果をCSVに保存 ===")
# 処理結果をCSVとして保存
output_path = "/home/glue_user/workspace/data/output/analysis_results"
joined_df.write.mode("overwrite").option("header", "true").csv(output_path)
print(f"結果を {output_path} に保存しました")

print("\n=== 処理完了 ===")
print("すべての処理が正常に完了しました！")

=== 1. Spark環境のセットアップ ===
Spark環境の準備完了

=== 2. CSVファイル直接読み込み ===
読み込みパス: /home/glue_user/workspace/data/warehouse/test_table/test_table.csv


CSVファイルの読み込み完了

=== 3. データの確認 ===
読み込んだデータ:
+---+----------+---+----------+
| id|      name|age|department|
+---+----------+---+----------+
|  1|テスト太郎| 30|    開発部|
|  2|テスト花子| 25|    営業部|
|  3|テスト次郎| 40|    総務部|
|  4|テスト三郎| 35|    人事部|
+---+----------+---+----------+

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- department: string (nullable = true)


=== 4. データ変換例 ===
文字列操作による変換:
+---+----------+---+----------+-------------------+
| id|      name|age|department|          full_info|
+---+----------+---+----------+-------------------+
|  1|テスト太郎| 30|    開発部|テスト太郎 (開発部)|
|  2|テスト花子| 25|    営業部|テスト花子 (営業部)|
|  3|テスト次郎| 40|    総務部|テスト次郎 (総務部)|
|  4|テスト三郎| 35|    人事部|テスト三郎 (人事部)|
+---+----------+---+----------+-------------------+

部門コードを追加:
+---+----------+---+----------+---------------+
| id|      name|age|department|department_code|
+---+----------+---+----------+---------------+
|  1|テスト太郎| 30|    開発部|             開|

+----------+---+----------+---+--------+
|department| id|      name|age|location|
+----------+---+----------+---+--------+
|    開発部|  1|テスト太郎| 30|    東京|
|    営業部|  2|テスト花子| 25|    大阪|
|    総務部|  3|テスト次郎| 40|  名古屋|
|    人事部|  4|テスト三郎| 35|    東京|
+----------+---+----------+---+--------+


=== 7. 結果をCSVに保存 ===
結果を /home/glue_user/workspace/data/output/analysis_results に保存しました

=== 処理完了 ===
すべての処理が正常に完了しました！
